In [16]:
import time
import pandas as pd
import requests
import shared
shared.init()
import json
from web3 import Web3, HTTPProvider
from hexbytes import HexBytes
from web3.datastructures import AttributeDict
from Utils.eth_utils import  obtain_hash_event

def connect_to_web3():
    """
    Connect to Web3 server.
    Args:
    Returns:
      res: Boolean indicating that connection was succeed or not.
      web3: Web3 Object
    """
    web3 = Web3(HTTPProvider('https://polygon-mainnet.infura.io/v3/d6243bb783b44485ad6636b6c3411377'))
    res = web3.isConnected()
    return res, web3

# from_block = 22757547
from_block = 11000000
response = [0] * 1000
block_number = []
_, web3 = connect_to_web3()
contract = web3.eth.contract(shared.SUSHISWAP_FACTORY, abi=shared.SUSHI_ABI_FACTORY)

#topic = "0x783cca1c0412dd0d695e784568c96da2e9c22ff989357a2e8b1d9b2b4e6b7118"
topic = obtain_hash_event('PairCreated(address,address,address,uint256)')


tokens0, tokens1, pairs, block_numbers, timestamps = [], [], [], [], []
cont = 1000
while len(response) >= 1000:
    try:
        print("Pools trobades", cont)
        cont += 1000
        apikey = "P6Z71M2VBHAHHBFMM91YT5YWENGJYTJT81"
        endpoint = "https://api.polygonscan.com/api?module=logs" \
                   "&action=getLogs" \
                   f"&fromBlock={from_block}" \
                   f"&toBlock={shared.BLOCKSTUDY}" \
                   f"&address={shared.SUSHISWAP_FACTORY}" \
                   f"&topic0={topic}" \
                   f"&apikey={apikey}"

        response = json.loads(requests.get(endpoint).text)['result']

        # print(response)
        receipts = []
        for event in response:
            receipts.append(AttributeDict({
                'blockNumber': int(event['blockNumber'], 16),
                'contractAddress': event['address'],
                'topics': [HexBytes(topic) for topic in event['topics']],
                'transactionHash': HexBytes(event['transactionHash']),
                'data': event['data'],
                'logIndex': 0,
                'transactionIndex': event['transactionIndex'],
                'address': event['address'],
                'blockHash': HexBytes(event['transactionHash'])
            }))
        log_dict = {'logs': receipts}
        # print(log_dict)
        decoded_logs = contract.events.PairCreated().processReceipt(log_dict)
        # print(decoded_logs)
        for k, log in enumerate(decoded_logs):
            block_numbers.append(log['blockNumber'])
            tokens0.append(log['args']["token0"])
            tokens1.append(log['args']['token1'])
            pairs.append(log['args']['pair'])
            #fees.append(log['args']['fee'])
            timestamps.append(int(response[k]['timeStamp'], 16))
        from_block = int(response[-1]['blockNumber'], 16) + 1

    except Exception as err:
        print("error", err)
        time.sleep(2)

pd.DataFrame({'pair': pairs, 'token0': tokens0, 'token1': tokens1, 'block_number': block_numbers, 'timestamp': timestamps}).to_csv(
    "./data/sushiswap/polygon_pools.csv", index=False)

Pools trobades 1000
Pools trobades 2000
Pools trobades 3000
Pools trobades 4000
Pools trobades 5000
Pools trobades 6000


In [56]:
import os
import time
import pandas as pd
import requests
import shared
from Utils.eth_utils import obtain_hash_event, connect_to_web3
shared.init()
import json
from hexbytes import HexBytes
from web3.datastructures import AttributeDict

def get_response(pool, from_block):
    endpoint = "https://api.polygonscan.com/api?module=logs" \
               "&action=getLogs" \
               f"&fromBlock={from_block}" \
               f"&toBlock={shared.BLOCKSTUDY}" \
               f"&address={pool}" \
               f"&topic0={sync_hash}" \
               f"&apikey={apikey}"

    return json.loads(requests.get(endpoint).text)['result']

def clean_response(response):
    receipts = []
    for event in response:
        receipts.append(AttributeDict({
            'blockNumber': int(event['blockNumber'], 16),
            'contractAddress': event['address'],
            'topics': [HexBytes(topic) for topic in event['topics']],
            'transactionHash': HexBytes(event['transactionHash']),
            'data': event['data'],
            'logIndex': 0,
            'transactionIndex': event['transactionIndex'],
            'address': event['address'],
            'blockHash': HexBytes(event['transactionHash'])
        }))

    log_dict = {'logs': receipts}
    return contract.events.Sync().processReceipt(log_dict)

sync_hash = obtain_hash_event('Sync(uint112,uint112)')
_, web3 = connect_to_web3()
apikey = "P6Z71M2VBHAHHBFMM91YT5YWENGJYTJT81"
tokens_and_pools = pd.read_csv("./data/sushiswap/polygon_pools.csv", index_col='pair')
pools = tokens_and_pools.index.tolist()

contract = web3.eth.contract(shared.EXCHANGES['sushiswap'], abi=shared.ABI_POOL)
done_pools = [pool.split(".csv")[0] for pool in os.listdir("./data/sushiswap/sync")]

for i, pool in enumerate(pools):
    if pool in done_pools:
        continue
    response = [0] * 1000

    print(pool, i, len(pools))
    timestamps, block_numbers, reserves0, reserves1 = [], [], [], []
    from_block, first_timestamp, current_timestamp = 0, 0, 0
    cont = 0

    while len(response) >= 1000 and current_timestamp - first_timestamp < shared.WEEK:
        response = get_response(pool, from_block)
        while response == None or "Max rate limit" in response:
            time.sleep(20)
            response = get_response(pool, from_block)
            if response != None:
                break
        if len(response) == 0:
            continue
        decoded_logs = clean_response(response)
        for k, log in enumerate(decoded_logs):
            block_numbers.append(log['blockNumber'])
            reserves0.append(log['args']["reserve0"])
            reserves1.append(log['args']["reserve1"])
            timestamps.append(int(response[k]["timeStamp"], 16))
        from_block = int(response[-1]['blockNumber'], 16) + 1
        current_timestamp = int(response[-1]['timeStamp'], 16)

    pd.DataFrame({'block_number': block_numbers, 'reseve0': reserves0 , 'reserve1': reserves1, 'timestamps': timestamps}).to_csv(
        f"./data/sushiswap/sync/{pool}.csv", index=False)

0xD1641d4ad34517ca20A64289025D9b18fab81cF7 3181 5782
0x0fA764e106D2f7860437FFd682E4488BEC7259a4 3182 5782
0xD272165BC7DBf967C7cE0aE9911940742f60cFfF 3183 5782
0xa2c2032366403bC26f3D04576aac53533f49516B 3184 5782
0xed4B9e9F916ff2b61D504833daB7E876aa6C2Ff0 3185 5782
0x69a139d6A63f42F399Cd37d04E412df516be0494 3186 5782
0xaBD1917C61d50211a5516D2337fb7A2f55715bd5 3187 5782
0x5BbBC9Bc6922985c93B35BE94005cFbae0cc703D 3188 5782
0x7648f54B0D6c69f05d2E815336E021c7E7960AB4 3189 5782
0x15ec73998E1E67B5f87164eAEBa4eE38A5ca606B 3190 5782
0xC71cd765166F1720720C06855D707DAefb53d79a 3191 5782
0x3c2646A14b43C872a09EC9dc6135Fc5920A50f56 3192 5782
0xa07335229e2B63E95188f3580Ee24ab4244fA7b0 3193 5782
0x978a3d6446802cBF70BF4b7AaA9ddD5889bE2b5A 3194 5782
0x623d9f9EeBEea66b101f0470f1d24DebaECF39ca 3195 5782
0x0a9d9EEEee45b82d96416c5C4f6b4Ad9104544e4 3196 5782
0x22B523b337D9c5d0cE132bAb9B230c6ab122E4B8 3197 5782
0xD04C202997C62bF54DD31298553C24D5a3Bce851 3198 5782
0xBb5317fC8939450B750Ed5c926fb3a67bbE6A0Ea 319

In [61]:
import os
import time
import pandas as pd
import requests
import shared
from Utils.eth_utils import obtain_hash_event, connect_to_web3
shared.init()
import json
from hexbytes import HexBytes
from web3.datastructures import AttributeDict

def get_response(pool, from_block):
    endpoint = "https://api.polygonscan.com/api?module=logs" \
               "&action=getLogs" \
               f"&fromBlock={from_block}" \
               f"&toBlock={shared.BLOCKSTUDY}" \
               f"&address={pool}" \
               f"&topic0={sync_hash}" \
               f"&apikey={apikey}"

    return json.loads(requests.get(endpoint).text)['result']

def clean_response(response):
    receipts = []
    for event in response:
        receipts.append(AttributeDict({
            'blockNumber': int(event['blockNumber'], 16),
            'contractAddress': event['address'],
            'topics': [HexBytes(topic) for topic in event['topics']],
            'transactionHash': HexBytes(event['transactionHash']),
            'data': event['data'],
            'logIndex': 0,
            'transactionIndex': event['transactionIndex'],
            'address': event['address'],
            'blockHash': HexBytes(event['transactionHash'])
        }))

    log_dict = {'logs': receipts}
    return contract.events.Burn().processReceipt(log_dict)

sync_hash = obtain_hash_event('Burn(address,uint256,uint256,address)')
_, web3 = connect_to_web3()
apikey = "P6Z71M2VBHAHHBFMM91YT5YWENGJYTJT81"
tokens_and_pools = pd.read_csv("./data/sushiswap/polygon_pools.csv", index_col='pair')
pools = tokens_and_pools.index.tolist()

contract = web3.eth.contract(shared.EXCHANGES['sushiswap'], abi=shared.ABI_POOL)
done_pools = [pool.split(".csv")[0] for pool in os.listdir("./data/sushiswap/burn")]

for i, pool in enumerate(pools):
    if pool in done_pools:
        continue
    response = [0] * 1000

    print(pool, i, len(pools))
    timestamps, block_numbers, sender, to, amount0, amount1 = [], [], [], [], [], []
    from_block, first_timestamp, current_timestamp = 0, 0, 0
    cont = 0       
    while len(response) >= 1000 and cont < 15:
        response = get_response(pool, from_block)
        while response == None or "Max rate limit" in response:
            time.sleep(20)
            response = get_response(pool, from_block)
            if response != None:
                break
        if len(response) == 0:
            continue
        try:
            decoded_logs = clean_response(response)
        except: 
            print(response)
        #print(decoded_logs)
        for k, log in enumerate(decoded_logs):
            block_numbers.append(log['blockNumber'])
            sender.append(log['args']["sender"])
            to.append(log['args']["to"])
            amount0.append(log['args']['amount0'])
            amount1.append(log['args']['amount1'])
            timestamps.append(int(response[k]["timeStamp"], 16))

        from_block = int(response[-1]['blockNumber'], 16) + 1
        current_timestamp = int(response[-1]['timeStamp'], 16)
        cont+=1
    pd.DataFrame({'block_number': block_numbers, 'sender': sender, "to":to,
                  'amount0':amount0,'amount1':amount1,'timestamps': timestamps
    }).to_csv(f"./data/sushiswap/burn/{pool}.csv", index=False)


0xc2FEC6e52A2e4622eb91E5ae4f23F0eA73c47Aa2 720 5782
0xe680C618c5cF1A0Ef6E7F18f8d4167093646ddf5 721 5782
0xAa5843865437ed4448BF95904f114264F0E4B113 722 5782
0x93819FeD6b6142e5Cf90D26495F1A05A45ad74a7 723 5782
0x8a8Cb7Bd7097688CA29a0E71B354B3BBe3Bb8AD4 724 5782
0x3168BBaC880F12aC072Ac062c17F637EDbc4a97D 725 5782
0xc5fA4EFE0E141BE9d6008b974806Cc0341423ED7 726 5782
0x286d63a6246d061FE93501C64b8Fa9A04216d009 727 5782
0x00fAE3b6FC7bCEe980C1E79A8b63a80ff6f6A0bE 728 5782
0xFA85f5bfcD5AC72cd95fFAd219D7787191D600ca 729 5782
0x01748ABbd18CFaf852E9149bAa05346825BAE2F5 730 5782
0xA8fD60af26dB6c7B8E834e9492B698b4531805b8 731 5782
0xFec2d4ed48f363f803c335503992144eD35ff51a 732 5782
0x3fB49917b9D18A13eC7465E448cEC600F29E5B4B 733 5782
0x46CdDd894Ea6e9e9C6E94b7fD11C48482483f05c 734 5782
0xa6f5CF768A0a9d0Ee712491Ee981778602b0728b 735 5782
0x9A63835170469273E31cDC5777aE8016f010033C 736 5782
0xe9Bb87c33d1607B06fb3988b31E9601BD21Ff3bD 737 5782
0xdD650C8d274474FF1af1152B3B27f2702AcA8a98 738 5782
0x7270b47733

In [67]:
import os
import time
import pandas as pd
import requests
import shared
from Utils.eth_utils import obtain_hash_event, connect_to_web3
shared.init()
import json
from hexbytes import HexBytes
from web3.datastructures import AttributeDict

def get_response(pool, from_block):
    endpoint = "https://api.polygonscan.com/api?module=logs" \
               "&action=getLogs" \
               f"&fromBlock={from_block}" \
               f"&toBlock={shared.BLOCKSTUDY}" \
               f"&address={pool}" \
               f"&topic0={sync_hash}" \
               f"&apikey={apikey}"

    return json.loads(requests.get(endpoint).text)['result']

def clean_response(response):
    receipts = []
    for event in response:
        receipts.append(AttributeDict({
            'blockNumber': int(event['blockNumber'], 16),
            'contractAddress': event['address'],
            'topics': [HexBytes(topic) for topic in event['topics']],
            'transactionHash': HexBytes(event['transactionHash']),
            'data': event['data'],
            'logIndex': 0,
            'transactionIndex': event['transactionIndex'],
            'address': event['address'],
            'blockHash': HexBytes(event['transactionHash'])
        }))

    log_dict = {'logs': receipts}
    return contract.events.Mint().processReceipt(log_dict)

sync_hash = obtain_hash_event("Mint(address,uint256,uint256)")
_, web3 = connect_to_web3()
apikey = "P6Z71M2VBHAHHBFMM91YT5YWENGJYTJT81"
tokens_and_pools = pd.read_csv("./data/sushiswap/polygon_pools.csv", index_col='pair')
pools = tokens_and_pools.index.tolist()

contract = web3.eth.contract(shared.EXCHANGES['sushiswap'], abi=shared.ABI_POOL)
done_pools = [pool.split(".csv")[0] for pool in os.listdir("./data/sushiswap/mint")]

for i, pool in enumerate(pools):
    if pool in done_pools:
        continue
    response = [0] * 1000

    print(pool, i, len(pools))
    timestamps, block_numbers, sender, amount0, amount1 = [], [], [], [], []
    from_block, first_timestamp, current_timestamp = 0, 0, 0
    cont = 0       
    while len(response) >= 1000 and cont < 15:
        response = get_response(pool, from_block)

        while response == None:
            time.sleep(20)
            response = get_response(pool, from_block)
            if response != None:
                break

        if len(response) == 0:
            continue
        decoded_logs = clean_response(response)
        
        # print(decoded_logs)
        for k, log in enumerate(decoded_logs):
            block_numbers.append(log['blockNumber'])
            sender.append(log['args']["sender"])
            amount0.append(log['args']['amount0'])
            amount1.append(log['args']['amount1'])
            timestamps.append(int(response[k]["timeStamp"], 16))

        from_block = int(response[-1]['blockNumber'], 16) + 1
        current_timestamp = int(response[-1]['timeStamp'], 16)
        cont+=1
    pd.DataFrame({'block_number': block_numbers, 'sender': sender, 
                  'amount0':amount0,'amount1':amount1,'timestamps': timestamps
    }).to_csv(f"./data/sushiswap/mint/{pool}.csv", index=False)

0xc4e595acDD7d12feC385E5dA5D43160e8A0bAC0E 0 5782
0x34965ba0ac2451A34a0471F04CCa3F990b8dea27 1 5782
0x8929D3FEa77398F64448c85015633c2d6472fB29 2 5782
0xf69e93771F11AECd8E554aA165C3Fe7fd811530c 3 5782
0x3054eD5F568B0ED3d1a506F64291b65da72B97F1 4 5782
0x68ccE7049013ca8df91CD512ceFEe8FC0bb8d926 5 5782
0xc4A681264F93B8DB5d554EFb4de6eE011Ada091b 6 5782
0xF8326C8aED2d21364fdD7D4ffc0C771755d8F55c 7 5782
0x7FAe0648684ec375176E2d0F171cA9814798D920 8 5782
0xcd353F79d9FADe311fC3119B841e1f456b54e858 9 5782
0x55FF76BFFC3Cdd9D5FdbBC2ece4528ECcE45047e 10 5782
0xC0C58dbC1De69a4A399965D3ec265E3A422bd700 11 5782
0x3122376d7128da298d115016658EF943d45334B6 12 5782
0x383Be588327216586e131E63592A2Dc976A16655 13 5782
0xf4F78fda908563331360B331130342867F74eb21 14 5782
0xe83f1c7990440445EE52f1395686A3e4BF5b4e47 15 5782
0x1D76935014Fe8F1179a5944154aa9c402f1b0837 16 5782
0xCD578F016888B57F1b1e3f887f392F0159E26747 17 5782
0x83fDCE3F2025aA58Bc99d89a43EE5D2bfEB74500 18 5782
0x8531c4e29491fE6e5e87AF6054FC20FcCf0b429